<a href="https://colab.research.google.com/github/harinivas04/Land/blob/main/LMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
!pip install tensorflow
!pip install tensorflow_hub
!pip install pillow
!pip install pandas
!pip install geopy
!pip install wikipedia-api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.5/93.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

In [2]:
import gradio as gr
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from geopy.geocoders import Nominatim
import folium
import requests
import wikipediaapi

In [3]:
TF_MODEL_URL= 'https://tfhub.dev/google/on_device_vision/classifier/landmarks_classifier_asia_V1/1'
LABEL_MAP_URL = 'https://www.gstatic.com/aihub/tfhub/labelmaps/landmarks_classifier_asia_V1_label_map.csv'
IMAGE_SHAPE = (321, 321)

In [4]:
GOOGLE_MAPS_API_KEY = 'your_google_maps_api_key_here'

In [5]:
try:
    classifier = tf.keras.Sequential( hub.KerasLayer(TF_MODEL_URL,input_shape=IMAGE_SHAPE + (3,), output_key="predictions:logits"))
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading the model: {e}")
try:
    df = pd.read_csv(LABEL_MAP_URL)
    label_map = dict(zip(df.id.astype(str), df.name))
    print("Label map loaded successfully.")
    print("Sample labels:", list(label_map.items())[:5])  # Print sample of label map
except Exception as e:
    print(f"Error loading the label map: {e}")

Error loading the model: 'KerasLayer' object is not iterable
Label map loaded successfully.
Sample labels: [('0', 'Bagha Shahi Mosque'), ('1', 'City Palace'), ('2', 'The Immaculate Conception Cathedral of Cubao'), ('3', 'City of David'), ('4', 'Noor Mahal')]


In [6]:
def classify_image(image):
    try:
        img = np.array(image) / 255.0  # Normalize the image
        img = tf.image.resize(img, IMAGE_SHAPE)  # Resize the image
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        predictions = classifier(img)
        predicted_label_id = np.argmax(predictions, axis=-1)[0]
        predicted_label = label_map.get(str(predicted_label_id), "Unknown")
        return predicted_label
    except Exception as e:
        print(f"Error in classify_image: {e}")
        return "Error"

In [7]:
def get_map_location(loc):
    try:
        geolocator = Nominatim(user_agent="landmark_recognition")
        location = geolocator.geocode(loc)
        if location:
            return location.address, location.latitude, location.longitude
        else:
            return "Location not found", None, None
    except Exception as e:
        print(f"Error in get_map_location: {e}")
        return "Error", None, None

In [8]:
def create_map(latitude, longitude):
    try:
        if latitude is not None and longitude is not None:
            m = folium.Map(location=[latitude, longitude], zoom_start=15)
            google_satellite_layer_folium.TileLayer(tiles=f"https://mt1.google.com/vt/lyrs=s&x={{x}}&y={{y}}&z={{z}}&apikey={GOOGLE_MAPS_API_KEY}",attr="Google",name="Google Satellite",overlay=True,control=True,position="topright")
            m.add_child(google_satellite_layer)
            folium.Marker([latitude, longitude], popup='Predicted Location').add_to(m)
            street_view_link=f'<ahref="https://www.google.com/maps?q=&layer=c&cbll={latitude},{longitude}" target="_blank">Street View link</a>'
            folium.Marker([latitude, longitude], popup=street_view_link).add_to(m)
            m.add_child(folium.LatLngPopup())  # Adds lat/lng popup on click
            m.add_child(folium.LayerControl())  # Adds layer control

            return m._repr_html_()
        else:
            return "Map not available"
    except Exception as e:
        print(f"Error in create_map: {e}")
        return "Error"

In [9]:
def fetch_landmark_info(landmark_name):
    try:
        wikipedia_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{landmark_name}"
        response = requests.get(wikipedia_url)
        if response.status_code == 200:
            data = response.json()
            summary = data.get('extract', 'No information available.')
            return summary
        else:
            return "Information not available."
    except Exception as e:
        print(f"Error fetching landmark info: {e}")
        return "Error"

In [10]:
def predict_landmark(*images):
    results = []
    for i, image in enumerate(images):
        try:
            img = Image.fromarray(image).convert("RGB")
            prediction = classify_image(img)
            address, latitude, longitude = get_map_location(prediction)
            map_html = create_map(latitude, longitude)
            street_view_link=f'<ahref="https://www.google.com/maps?q=&layer=c&cbll={latitude},{longitude}" target="_blank" style="color: blue; text-decoration: underline;">Street View link</a>'
            landmark_info = fetch_landmark_info(prediction)
            results.extend([prediction,address,{'Latitude': latitude, 'Longitude': longitude},map_html,street_view_link, landmark_info ])
        except Exception as e:
            print(f"Error in predict_landmark for image {i+1}: {e}")
            results.extend(["Error", "Error", {"Latitude": None, "Longitude": None}, "Error", "Error", "Error"])
    return results

In [11]:
def predict_landmark(*images):
    results = []
    for i, image in enumerate(images):
        try:
            # Convert the image to RGB format if needed
            img = Image.fromarray(image).convert("RGB")

            # Classify the image
            prediction = classify_image(img)

            # Get map location info based on the prediction
            address, latitude, longitude = get_map_location(prediction)

            # Generate the map HTML
            map_html = create_map(latitude, longitude)

            # Create the Street View link
            street_view_link = f'<a href="https://www.google.com/maps?q=&layer=c&cbll={latitude},{longitude}" target="_blank" style="color: blue; text-decoration: underline;">Street View link</a>'

            # Fetch additional landmark info
            landmark_info = fetch_landmark_info(prediction)

            # Append results for this image
            results.extend([
                prediction,
                address,
                {'Latitude': latitude, 'Longitude': longitude},
                map_html,
                street_view_link,
                landmark_info
            ])
        except Exception as e:
            print(f"Error in predict_landmark for image {i+1}: {e}")
            results.extend([
                "Error",
                "Error",
                {"Latitude": None, "Longitude": None},
                "Error",
                "Error",
                "Error"
            ])

    return results


In [12]:
inputs = [gr.Image(image_mode='RGB', type='numpy', label=f"Image {i+1}") for i in range(3)]
title = "Landmark Recognition with Enhanced Google Maps and Street View"
description = "Upload up to three images and get the predicted landmarks along with their addresses, interactive maps with Google Maps satellite view, and street view links. You'll also get information about the landmark's history, culture, and more."

outputs = [
    gr.Textbox(label="Predicted Landmark 1"),
    gr.Textbox(label="Address 1"),
    gr.JSON(label="Latitude & Longitude 1"),
    gr.HTML(label="Map 1"),
    gr.HTML(label="Street View Link 1"),
    gr.Textbox(label="Landmark Info 1"),  # Additional output for landmark information
    gr.Textbox(label="Predicted Landmark 2"),
    gr.Textbox(label="Address 2"),
    gr.JSON(label="Latitude & Longitude 2"),
    gr.HTML(label="Map 2"),
    gr.HTML(label="Street View Link 2"),
    gr.Textbox(label="Landmark Info 2"),  # Additional output for landmark information
    gr.Textbox(label="Predicted Landmark 3"),
    gr.Textbox(label="Address 3"),
    gr.JSON(label="Latitude & Longitude 3"),
    gr.HTML(label="Map 3"),
    gr.HTML(label="Street View Link 3"),
    gr.Textbox(label="Landmark Info 3"),  # Additional output for landmark information
]
gr.Interface(fn=predict_landmark,inputs=inputs,outputs=outputs,title=title,description=description).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9ecc593ab7dc5f0e37.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [13]:
import pandas as pd
import requests
from io import StringIO

# URL of the label map CSV file
LABEL_MAP_URL = 'https://www.gstatic.com/aihub/tfhub/labelmaps/landmarks_classifier_asia_V1_label_map.csv'

def download_label_map(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            # Read CSV from response content
            csv_data = response.content.decode('utf-8')
            # Create a pandas DataFrame
            df = pd.read_csv(StringIO(csv_data))
            return df
        else:
            print(f"Failed to download label map. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error downloading label map: {e}")
        return None

# Download the label map
label_map_df = download_label_map(LABEL_MAP_URL)

# Example: Print sample labels
if label_map_df is not None:
    print("Label map loaded successfully.")
    print("Sample labels:")
    print(label_map_df.head())
else:
    print("Failed to load label map.")

Label map loaded successfully.
Sample labels:
   id                                          name
0   0                            Bagha Shahi Mosque
1   1                                   City Palace
2   2  The Immaculate Conception Cathedral of Cubao
3   3                                 City of David
4   4                                    Noor Mahal


In [14]:
def load_model_and_label_map():
    try:
        # Load the model
        model = tf.keras.Sequential([
            hub.KerasLayer(TF_MODEL_URL, input_shape=IMAGE_SHAPE + (3,), output_key="predictions:logits")
        ])
        print("Model loaded successfully.")

        # Load the label map
        df = pd.read_csv(LABEL_MAP_URL)
        label_map = dict(zip(df['id'].astype(str), df['name']))
        print("Label map loaded successfully.")

        return model, label_map
    except Exception as e:
        print(f"Error loading model or label map: {e}")
        return None, None

# Function to classify a single image
def classify_image(image, model):
    try:
        img = np.array(image) / 255.0  # Normalize the image
        img = tf.image.resize(img, IMAGE_SHAPE)  # Resize the image
        img = np.expand_dims(img, axis=0)  # Add batch dimension
        predictions = model.predict(img)
        predicted_label_id = np.argmax(predictions, axis=-1)[0]
        return predicted_label_id
    except Exception as e:
        print(f"Error in classify_image: {e}")
        return None

# Function to predict accuracy
def predict_accuracy(test_images_dir, ground_truth_file):
    try:
        # Load model and label map
        model, label_map = load_model_and_label_map()
        if model is None or label_map is None:
            return

        # Load ground truth annotations
        ground_truth = pd.read_csv(ground_truth_file)

        total_images = len(ground_truth)
        correct_predictions = 0

        for index, row in ground_truth.iterrows():
            image_filename = row['Image Filename']
            correct_landmark_id = str(row['Correct Landmark'])

            # Load and classify the image
            image_path = os.path.join(test_images_dir, image_filename)
            image = Image.open(image_path).convert('RGB')
            predicted_label_id = classify_image(image, model)
            predicted_landmark_id = str(predicted_label_id)

            # Compare predicted with ground truth
            if predicted_landmark_id == correct_landmark_id:
                correct_predictions += 1

        # Calculate accuracy
        accuracy = (correct_predictions / total_images) * 100
        print(f"Accuracy: {accuracy:.2f}%")

    except Exception as e:
        print(f"Error predicting accuracy: {e}")

# Example usage
test_images_dir = "/content/taj.jpg"
ground_truth_file = "https://www.gstatic.com/aihub/tfhub/labelmaps/landmarks_classifier_asia_V1_label_map.csv"
predict_accuracy(test_images_dir, ground_truth_file)

Error loading model or label map: Only instances of `keras.Layer` can be added to a Sequential model. Received: <tensorflow_hub.keras_layer.KerasLayer object at 0x7d24daad3dc0> (of type <class 'tensorflow_hub.keras_layer.KerasLayer'>)
